In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install nupunkt -q

import ast
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nupunkt import sent_tokenize

import torch
from transformers import AutoTokenizer, AutoModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.2 MB/s eta 0:00:00


In [3]:
MODEL_NAME = "answerdotai/ModernBERT-base"

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
MODEL = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [4]:
df = pd.read_json('/content/drive/MyDrive/metadata/val.json')

df.head()

,case_id,clean_content,chunks,num_chunks,label
0,3,"The case ""David B. v. McDonald"" from Illinois ...","['The case ""David B. v. McDonald"" from Illinoi...",10,0
1,16,"The case ""Harrelson v. Elmore County"" from Ala...","['The case ""Harrelson v. Elmore County"" from A...",1,0
2,24,"The case ""Hill v. Pulaski County"" from Arkansa...","['The case ""Hill v. Pulaski County"" from Arkan...",1,0
3,29,"The case ""U.S. v. Maricopa County"" from Arizon...","['The case ""U.S. v. Maricopa County"" from Ariz...",1,0
4,146,"The case ""Tyler v. Percich"" from Missouri was ...","['The case ""Tyler v. Percich"" from Missouri wa...",2,0


In [5]:
len(df)

800

In [6]:
MODEL.to('cuda')

BATCH_SIZE = 12

for index, row in tqdm(df.iterrows(), total=len(df)):
    case_id = row["case_id"]
    #print(f"Processing: ", index)

    chunks = ast.literal_eval(row["chunks"])

    all_embeddings = []

    for i in range(0, len(chunks), BATCH_SIZE):
        batch = chunks[i:i + BATCH_SIZE]

        # Tokenize and move inputs to GPU
        inputs = TOKENIZER(batch, max_length=8192, padding=True, truncation=True, return_tensors='pt')
        inputs = {k: v.to('cuda') for k, v in inputs.items()}

        # Run model on batch
        with torch.no_grad():
            outputs = MODEL(**inputs)
            last_hidden_state = outputs.last_hidden_state  # shape: [batch, seq_len, hidden_dim]
            #print("lhs: ", last_hidden_state.shape)

            mean_embeddings = last_hidden_state.mean(dim=1)  # shape: [batch, hidden_dim]
            all_embeddings.append(mean_embeddings.cpu())

        torch.cuda.empty_cache()

    # Combine all embeddings: [n_chunks, hidden_dim]
    all_embeddings_tensor = torch.cat(all_embeddings, dim=0)
    final_embedding = all_embeddings_tensor.mean(dim=0).numpy()
    #print("embed: ", final_embedding.shape)

    df.at[index, 'mean_embedding'] = final_embedding.astype(object)

    torch.cuda.empty_cache()

100%|██████████| 800/800 [3:13:41<00:00, 14.53s/it]


In [7]:
#df["max_embedding"] = df["max_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df["mean_embedding"] = df["mean_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

df.to_json('/content/drive/MyDrive/metadata/embeddings/val_embeddings.json')

In [8]:
df.head()

,case_id,clean_content,chunks,num_chunks,label,mean_embedding
0,3,"The case ""David B. v. McDonald"" from Illinois ...","['The case ""David B. v. McDonald"" from Illinoi...",10,0,"[-0.6259282231330872, -0.40986889600753784, -0..."
1,16,"The case ""Harrelson v. Elmore County"" from Ala...","['The case ""Harrelson v. Elmore County"" from A...",1,0,"[-0.45644375681877136, -0.0400216318666935, 0...."
2,24,"The case ""Hill v. Pulaski County"" from Arkansa...","['The case ""Hill v. Pulaski County"" from Arkan...",1,0,"[-0.31623971462249756, -0.18306639790534973, -..."
3,29,"The case ""U.S. v. Maricopa County"" from Arizon...","['The case ""U.S. v. Maricopa County"" from Ariz...",1,0,"[-0.3785632252693176, -0.08938245475292206, -0..."
4,146,"The case ""Tyler v. Percich"" from Missouri was ...","['The case ""Tyler v. Percich"" from Missouri wa...",2,0,"[-0.7671036720275879, -0.4180658459663391, 0.0..."
